<a href="https://colab.research.google.com/github/Anirudh-020505/Solar-Energy-Forecasting/blob/main/solarForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

# 1. Mount the Drive (Follow the popup instructions)
drive.mount('/content/drive')

# 2. Define the path (Since they added a shortcut, the path is the same for everyone!)
# Note: Ensure the folder name in My Drive is exactly 'Solar_Project_Data'
path = '/content/drive/My Drive/Solar-ForeCast-Resource/'

# 3. Load the Data
gen_df = pd.read_csv(path + 'spg.csv')

print("Data Loaded Successfully!")
print(gen_df.head())

Mounted at /content/drive
Data Loaded Successfully!
   temperature_2_m_above_gnd  relative_humidity_2_m_above_gnd  \
0                       2.17                               31   
1                       2.31                               27   
2                       3.65                               33   
3                       5.82                               30   
4                       7.73                               27   

   mean_sea_level_pressure_MSL  total_precipitation_sfc  snowfall_amount_sfc  \
0                       1035.0                      0.0                  0.0   
1                       1035.1                      0.0                  0.0   
2                       1035.4                      0.0                  0.0   
3                       1035.4                      0.0                  0.0   
4                       1034.4                      0.0                  0.0   

   total_cloud_cover_sfc  high_cloud_cover_high_cld_lay  \
0                